In [0]:
import importlib
import pandas as pd
import os
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, DoubleType
import time
from datetime import datetime
from io import StringIO

In [0]:
%pip install pysurveycto
import pysurveycto as pcto

## Connect to Survey CTO

In [0]:
%pip install pysurveycto
import pysurveycto as pcto

def connect_survey_cto(): 
    dbutils.widgets.text("SURVEY_CTO_SERVER", "", "SURVEY_CTO_SERVER")
    dbutils.widgets.text("SURVEY_CTO_USERNAME", "", "SURVEY_CTO_USERNAME")
    dbutils.widgets.text("SURVEY_CTO_PASSWORD", "", "SURVEY_CTO_PASSWORD")

    server_name = dbutils.widgets.get("SURVEY_CTO_SERVER")
    username = dbutils.widgets.get("SURVEY_CTO_USERNAME")
    password = dbutils.widgets.get("SURVEY_CTO_PASSWORD")

    scto = pcto.SurveyCTOObject(server_name, username, password)
    return scto

## Get all Form Names

In [0]:
scto = connect_survey_cto()
all_forms = scto.list_forms()
all_ids = [form['id'] for form in all_forms]

In [0]:
all_forms

In [0]:
all_ids

## Pull & Write All Survey CTO Data

In [0]:
# Write to prg_mega
scto = connect_survey_cto()
def table_to_catalog(TABLE, df):
    CATALOG = "prd_mega"
    SCHEMA = "ssurve38"

    sdf = spark.createDataFrame(df)
    spark.sql(f"USE {CATALOG}.{SCHEMA}")
    sdf.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(TABLE)

    return

def attachments_to_catalog(form_id, attachments):
    CATALOG = 'prd_mega'
    SCHEMA  = 'ssurve38'
    VOLUME  = 'vsurve38'

    for uuid, fields in attachments.items():
        uuid_clean = uuid.split(':')[-1]
        base_path = f'/Volumes/{CATALOG}/{SCHEMA}/{VOLUME}'
        survey_path = os.path.join(base_path, 'Surveys')
        form_path = os.path.join(survey_path, form_id)
        uuid_path = os.path.join(form_path, uuid)
        attach_path = os.path.join(uuid_path, 'Attachments')
        

        if not os.path.exists(base_path):
            os.mkdir(base_path)
        if not os.path.exists(survey_path):
            os.mkdir(survey_path)
        if not os.path.exists(form_path):
            os.mkdir(form_path)
        if not os.path.exists(uuid_path):
            os.mkdir(uuid_path)
        if not os.path.exists(attach_path):
            os.mkdir(attach_path)

        for field, url in fields.items():
            field_path = os.path.join(attach_path, field)
            if not os.path.exists(field_path):
                os.mkdir(field_path)
            
            upload_path = os.path.join(field_path, url.split('/')[-1])

            attachment = scto.get_attachment(url)
            with open(upload_path, "wb") as f:
                f.write(attachment)

def get_form_data(form_id, format='json', oldest_completion_date=datetime(2004, 1, 1)):
    form_data = scto.get_form_data(form_id, format=format, oldest_completion_date=oldest_completion_date)
    
    return form_data

def get_dataset_data(dataset_id, format='json'):
    dataset_data = scto.get_server_dataset(dataset_id=dataset_id)

    return dataset_data

def get_attachment_urls(form_data):
    attachment_urls = {}

    for record in form_data:
        uuid = record['instanceID']
        attachment_base_url = f'https://{server_name}.surveycto.com/api/v2/forms/{form_id}/submissions/{uuid}/'
        for field, value in record.items():
            if attachment_base_url in value:
                if not uuid in attachment_urls.keys():
                    attachment_urls[uuid] = {}
                attachment_urls[uuid][field] = value

    return attachment_urls

def write_data(_id, df, attach_urls={}, num_pass=0, num_fail=0):
    try:
        table_to_catalog(_id, df)
        if len(attach_urls) > 0:
            attachments_to_catalog(_id, attachment_urls)
        print(_id, ': PASS')
        num_pass += 1
    except:
        num_fail += 1
        print(_id, ': FAIL (DB Upload)')
    
    return num_pass, num_fail

In [0]:
dataset_ids = ['dg_regio_ma_policy_survey_dataset']
num_pass = 0
num_fail = 0

line_breaks = '\n'
for dataset_id in dataset_ids:
    
    dataset_data = scto.get_server_dataset(dataset_id=dataset_id, line_breaks=line_breaks)
    csv_io = StringIO(dataset_data.strip())
    df = pd.read_csv(csv_io)

    num_pass, num_fail = write_data(dataset_id, df, num_pass=num_pass, num_fail=num_fail)

df

In [0]:
import pandas as pd
from io import StringIO

# Remove any leading/trailing whitespace and load as a file-like object

# Show first few rows
print(df.head())


In [0]:
# good_ids = ['dime_stc_application_mar2024'] # testing using a smaller subset of form id's 
good_ids = []

if good_ids == []:
    good_ids = all_ids

for form_id in good_ids:
    try:
        form_data = get_form_data(form_id)
        attachment_urls = get_attachment_urls(form_data)
        form_df = pd.DataFrame(form_data)
        num_pass, num_fail = write_data(form_id, form_df, attachment_urls, num_pass, num_fail)
    except:
        num_fail += 1

print('num_passed: ',num_pass)
print('num_failed: ',num_fail)